## Most Popular

### Configuración Inicial

In [1]:
!pip uninstall -y numpy
!pip install numpy==1.26

Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Successfully uninstalled numpy-2.0.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.5/58.5 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.9/17.9 MB 44.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jaxlib 0.7.2 requires numpy>=2.0, but you have numpy 1.26.0 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.0 which is incompatible.
jax 0.7.2 requires numpy>=2.0, but you have numpy 1.26.0 which is incompatible.
pytensor 2.35.1 requires numpy>=2.0, but you have numpy 1.26.0 which is incompatible.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.0 which is incompatible.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>

In [1]:
!pip install scikit-surprise --no-build-isolation --no-deps
!pip install memory_profiler

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp312-cp312-linux_x86_64.whl size=2708550 sha256=1d7590efa30af433874347dea6a3dc047db20ba3d9412296aae96d03083e2603
  Stored in directory: /root/.cache/pip/wheels/75/fa/bc/739bc2cb1fbaab6061854e6cfbb81a0ae52c92a502a7fa454b
Successfully built scikit-surprise


### Instalación de Librerías

In [2]:
import pandas as pd
import gdown
import time
from memory_profiler import memory_usage
from sklearn.metrics import mean_squared_error, mean_absolute_error
from math import sqrt

### Importación de los Datos

In [3]:
gdown.download(id='1H_24ycns6zbOVfHFJRI9vGjVffVA5z6v', output='training_ratings.csv', quiet=False)
gdown.download(id='1pKmf07ehHOmlvIyT8nv__vPuWE2Z3ygZ', output='validation_ratings.csv', quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=1H_24ycns6zbOVfHFJRI9vGjVffVA5z6v
From (redirected): https://drive.google.com/uc?id=1H_24ycns6zbOVfHFJRI9vGjVffVA5z6v&confirm=t&uuid=f083f8b5-7a31-48fd-a564-9c2d4c87ecbc
To: /content/training_ratings.csv
100%|██████████| 249M/249M [00:03<00:00, 63.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=1pKmf07ehHOmlvIyT8nv__vPuWE2Z3ygZ
To: /content/validation_ratings.csv
100%|██████████| 58.3M/58.3M [00:00<00:00, 63.8MB/s]


'validation_ratings.csv'

In [4]:
df_train = pd.read_csv('training_ratings.csv')
df_val = pd.read_csv('validation_ratings.csv')

### Preprocesamiento de Datos

In [5]:
df_train.drop_duplicates(inplace=True, subset=['user', 'item'])
df_val.drop_duplicates(inplace=True, subset=['user', 'item'])

In [6]:
item_avg_ratings = df_train.groupby('item')['rating'].mean()

item_avg_ratings_dict = item_avg_ratings.to_dict()

global_avg_rating = df_train['rating'].mean()
print(f"Se calculó el promedio para {len(item_avg_ratings)} ítems.")
print(f"El rating promedio global es: {global_avg_rating:.2f}")

Se calculó el promedio para 21925 ítems.
El rating promedio global es: 7.15


In [7]:
predictions = df_val['item'].map(item_avg_ratings_dict).fillna(global_avg_rating)

In [8]:
mse = mean_squared_error(df_val['rating'], predictions)
mae = mean_absolute_error(df_val['rating'], predictions)
rmse = mse ** (1/2)
print(f"\nEl Error Cuadrático Medio (RMSE) del modelo Most Popular es: {rmse:.4f}")
print(f"El Error Absoluto Medio (MAE) del modelo Most Popular es: {mae:.4f}")


El Error Cuadrático Medio (RMSE) del modelo Most Popular es: 1.3201
El Error Absoluto Medio (MAE) del modelo Most Popular es: 0.9992


In [9]:
def evaluar_most_popular_rmse(df_train, df_val):
    """
    Entrena, predice y evalúa el modelo Most Popular, retornando el RMSE.
    """
    # Entrenamiento
    item_avg_ratings = df_train.groupby('item')['rating'].mean().to_dict()
    global_avg_rating = df_train['rating'].mean()

    # Predicción
    predictions = df_val['item'].map(item_avg_ratings).fillna(global_avg_rating)

    # Evaluación
    mse = mean_squared_error(df_val['rating'], predictions)
    mae = mean_absolute_error(df_val['rating'], predictions)
    rmse = mse ** (1/2)

    return rmse, mae

start_time = time.time()
rmse_resultado, mae_resultado = evaluar_most_popular_rmse(df_train, df_val)
end_time = time.time()
elapsed_time = end_time - start_time

memoria_usada = memory_usage((evaluar_most_popular_rmse, (df_train, df_val)))
memoria_max = max(memoria_usada) - min(memoria_usada)

print(f"RMSE: {rmse_resultado:.4f}")
print(f"MAE: {mae_resultado:.4f}")
print(f"Tiempo de ejecución: {elapsed_time:.2f} segundos")
print(f"Memoria utilizada (pico): {memoria_max:.2f} MB")

RMSE: 1.3201
MAE: 0.9992
Tiempo de ejecución: 0.56 segundos
Memoria utilizada (pico): 94.59 MB


Ahora haremos lo mismo pero para un grupo de 4 personas que han calificado ese item (haremos 4 personas por mas que haya juegos que es de máximo 2 o 10 etc por simplicidad). Lo que haremos es ver cómo se compara la recomendación (modelo most popular) versus su promedio de calificaciones que le dieron al item.

Para este nos basamos en lo hecho en random pa

In [10]:
def evaluar_mostpopular_topn_grupos(df_train, df_val, n=10):
    """
    Genera recomendaciones tipo Most Popular para grupos de 4 usuarios.
    Para cada grupo se calculan candidatos (items no vistos por NADIE del grupo)
    y se puntúan por rating promedio del item (popularidad).
    Devuelve: { (u1,u2,u3,u4): [(item, score), ...] }
    """
    # Popularidad por item (promedio de rating) y promedio global
    item_avg = df_train.groupby('item')['rating'].mean().to_dict()
    global_avg = df_train['rating'].mean()

    # Items disponibles y "vistos" por usuario en train
    all_items = df_train['item'].unique().tolist()
    user2seen = df_train.groupby('user')['item'].apply(set).to_dict()

    # Armar grupos secuenciales de 4 a partir de los usuarios en validación
    usuarios = df_val['user'].unique().tolist()
    grupos = [tuple(usuarios[i:i+4])
              for i in range(0, len(usuarios) - (len(usuarios) % 4), 4)]

    top_n_grupo = {}

    for grupo in grupos:
        # Unión de items vistos por cualquiera del grupo
        seen_union = set()
        for uid in grupo:
            seen_union |= user2seen.get(uid, set())

        # Candidatos: no vistos por nadie del grupo
        candidates = list(set(all_items) - seen_union)
        if not candidates:
            top_n_grupo[grupo] = []
            continue

        # Puntuar por popularidad (fallback al promedio global si no hay info)
        scores = [(iid, item_avg.get(iid, global_avg)) for iid in candidates]

        # Ordenar por score descendente y tomar top-n
        scores.sort(key=lambda x: x[1], reverse=True)
        top_n_grupo[grupo] = scores[:n]

    return top_n_grupo


def rmse_mae_from_topn_grupo(top_n_grupo, df_val):
    """
    Calcula RMSE/MAE comparando las predicciones del grupo (score del item)
    con los ratings reales en df_val para miembros del grupo.
    Nota: imita la lógica de 'random' tomando los ratings disponibles del grupo.
    """
    real, predicho = [], []

    total = sum(len(recs) for recs in top_n_grupo.values())
    i = 0

    for grupo, recs in top_n_grupo.items():
        for iid, pred in recs:
            # Tomamos TODOS los ratings disponibles en el grupo para ese item
            real_vals = df_val.loc[
                (df_val['user'].isin(grupo)) & (df_val['item'] == iid),
                'rating'
            ].values

            # Si hay varios miembros con rating para ese item, contamos todos
            for rv in real_vals:
                real.append(rv)
                predicho.append(pred)

            i += 1
            if total > 0 and (i % 10000 == 0 or i == total):
                progreso = (i / total) * 100
                print(f"Progreso: {i}/{total} ({progreso:.2f}%)")

    # Si no hubo pares comparables, devolvemos NaN
    if len(real) == 0:
        return float('nan'), float('nan')

    return sqrt(mean_squared_error(real, predicho)), mean_absolute_error(real, predicho)


# ====== Ejecución (igual al estilo de las últimas celdas de random) ======
start_time = time.time()
top_n_grupo = evaluar_mostpopular_topn_grupos(df_train, df_val, n=10)
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Tiempo de ejecución (generación top-n grupos): {elapsed_time:.2f} segundos")

memoria = memory_usage(
    (evaluar_mostpopular_topn_grupos, (df_train, df_val), {'n':10})
)
print("Memoria usada (MB):", max(memoria) - min(memoria))

# Métricas
rmse_grupo, mae_grupo = rmse_mae_from_topn_grupo(top_n_grupo, df_val)
print("RMSE grupos (Most Popular):", rmse_grupo)
print("MAE grupos (Most Popular):", mae_grupo)

Tiempo de ejecución (generación top-n grupos): 1712.56 segundos
Memoria usada (MB): 965.578125
Progreso: 10000/651170 (1.54%)
Progreso: 20000/651170 (3.07%)


KeyboardInterrupt: 

In [11]:
import numpy as np
import pandas as pd
from sklearn.metrics import ndcg_score

print("Creando DataFrame de evaluación Most Popular...")

df_eval = df_val.copy()

df_eval = df_eval.rename(columns={'user': 'userID', 'item': 'itemID'})

df_eval['label'] = (df_eval['rating'] >= 7).astype(int)

df_eval['score'] = df_eval['itemID'].map(item_avg_ratings_dict).fillna(global_avg_rating)

print("DataFrame de evaluación (Most Popular) creado con éxito.")
print(df_eval.head())

print("\nCreando grupos sintéticos...")
user_counts = df_eval['userID'].value_counts()
valid_users = user_counts[user_counts >= 10].index.tolist()

np.random.seed(42)
num_groups = 1000
group_size = 4
groups = [np.random.choice(valid_users, group_size, replace=False) for _ in range(num_groups)]

print(f"Se crearon {len(groups)} grupos sintéticos de tamaño {group_size}.")
print("Ejemplo de un grupo:", groups[0])

print("\nAgregando predicciones para cada grupo...")
all_group_recs = []

for group_id, user_ids in enumerate(groups):
    group_predictions = df_eval[df_eval['userID'].isin(user_ids)]

    item_scores_per_group = group_predictions.groupby('itemID').agg(
        avg_score=('score', 'mean'),
        min_score=('score', 'min'),
        max_score=('score', 'max'),

        group_label=('label', lambda x: 1 if all(x == 1) else 0)
    ).reset_index()

    item_scores_per_group['group_id'] = group_id
    all_group_recs.append(item_scores_per_group)

df_group_eval = pd.concat(all_group_recs, ignore_index=True)
print("Agregación completada.")
print(df_group_eval.head())

def precision_recall_at_k(group, k):
    """Calcula Precision@K y Recall@K para un solo grupo."""
    group = group.sort_values('score', ascending=False)
    topk = group.head(k)
    hits = topk['label'].sum()
    total_relevant = group['label'].sum()
    precision = hits / k if k > 0 else 0
    recall = hits / total_relevant if total_relevant > 0 else 0
    return precision, recall

def ndcg_at_k(group, k):
    """Calcula nDCG@K para un solo grupo."""
    if group['label'].sum() == 0:
        return 0.0
    ranked_group = group.sort_values('score', ascending=False).head(k)
    if len(ranked_group) < 2:
        return 0.0
    true_relevance = np.asarray([ranked_group['label'].values])
    predicted_scores = np.asarray([ranked_group['score'].values])
    return ndcg_score(true_relevance, predicted_scores)

strategies = {
    'Average': 'avg_score',
    'Least Misery': 'min_score',
    'Most Pleasure': 'max_score'
}

group_results = []
K_values = [10]

for strategy_name, score_column in strategies.items():
    print(f"\nEvaluando estrategia (Most Popular): {strategy_name}...")
    df_strategy_eval = df_group_eval[['group_id', 'itemID', 'group_label']].copy()
    df_strategy_eval.rename(columns={'group_label': 'label'}, inplace=True)
    df_strategy_eval['score'] = df_group_eval[score_column]

    grouped_strategy = df_strategy_eval.groupby('group_id')

    for k in K_values:
        metrics = grouped_strategy.apply(lambda x: precision_recall_at_k(x, k))
        avg_precision = np.mean([m[0] for m in metrics])
        avg_recall = np.mean([m[1] for m in metrics])
        ndcg_scores = grouped_strategy.apply(lambda x: ndcg_at_k(x, k))
        avg_ndcg = np.mean(ndcg_scores)
        group_results.append({
            'Model': 'Most Popular',
            'Strategy': strategy_name,
            'K': k,
            'Precision@K': avg_precision,
            'Recall@K': avg_recall,
            'nDCG@K': avg_ndcg
        })

group_results_df = pd.DataFrame(group_results)

print("\n--- Resultados de Evaluación Grupal para Most Popular ---")
print(group_results_df)

Creando DataFrame de evaluación Most Popular...
DataFrame de evaluación (Most Popular) creado con éxito.
   itemID  rating     userID  label     score
0   18602     9.0   beastvol      1  7.819848
1     278     6.5    mycroft      0  6.761880
2       5     8.0    mycroft      1  7.449898
3  264295     7.0   -=Yod@=-      1  7.095217
4   38949     6.0   -=Yod@=-      0  5.672661

Creando grupos sintéticos...
Se crearon 1000 grupos sintéticos de tamaño 4.
Ejemplo de un grupo: ['Sidewynnder' 'razorclovis' 'dougielb' 'sandro']

Agregando predicciones para cada grupo...
Agregación completada.
   itemID  avg_score  min_score  max_score  group_label  group_id
0      50   7.249054   7.249054   7.249054            1         0
1      71   7.587081   7.587081   7.587081            1         0
2     171   7.278967   7.278967   7.278967            1         0
3     258   5.733879   5.733879   5.733879            1         0
4     438   6.576676   6.576676   6.576676            1         0

Evaluand

/tmp/ipython-input-3414664236.py:90: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  metrics = grouped_strategy.apply(lambda x: precision_recall_at_k(x, k))
/tmp/ipython-input-3414664236.py:93: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ndcg_scores = grouped_strategy.apply(lambda x: ndcg_at_k(x, k))



Evaluando estrategia (Most Popular): Least Misery...


/tmp/ipython-input-3414664236.py:90: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  metrics = grouped_strategy.apply(lambda x: precision_recall_at_k(x, k))
/tmp/ipython-input-3414664236.py:93: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ndcg_scores = grouped_strategy.apply(lambda x: ndcg_at_k(x, k))



Evaluando estrategia (Most Popular): Most Pleasure...


/tmp/ipython-input-3414664236.py:90: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  metrics = grouped_strategy.apply(lambda x: precision_recall_at_k(x, k))



--- Resultados de Evaluación Grupal para Most Popular ---
          Model       Strategy   K  Precision@K  Recall@K    nDCG@K
0  Most Popular        Average  10       0.9017  0.143909  0.967632
1  Most Popular   Least Misery  10       0.9017  0.143909  0.967632
2  Most Popular  Most Pleasure  10       0.9017  0.143909  0.967632


/tmp/ipython-input-3414664236.py:93: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ndcg_scores = grouped_strategy.apply(lambda x: ndcg_at_k(x, k))
